# import 

In [3]:

from pathlib import Path, PurePosixPath
from os.path import abspath
from loguru import logger
import sys
import os
import numpy as np
import pickle
import os
from rich.progress import (
    Progress,
    BarColumn,
    TextColumn,
    ProgressColumn,
)
from rich.text import Text
from datetime import timedelta
from myterial import (
    orange,
    teal_light,
    light_blue_light,
)

class TimeRemainingColumn(ProgressColumn):
    """Renders estimated time remaining."""

    _table_column = None
    # Only refresh twice a second to prevent jitter
    max_refresh = 0.5

    def render(self, task):
        """Show time remaining."""
        remaining = task.time_remaining
        if remaining is None:
            return Text("-:--:--", style=teal_light)
        remaining_delta = timedelta(seconds=int(remaining))
        return Text("remaining: " + str(remaining_delta), style=teal_light)


class TimeElapsedColumn(ProgressColumn):
    """Renders estimated time elapsed."""

    _table_column = None
    # Only refresh twice a second to prevent jitter
    max_refresh = 0.5

    def render(self, task):
        """Show time elapsed."""
        elapsed = task.elapsed
        if elapsed is None:
            return Text("-:--:--", style=light_blue_light)
        elapsed_delta = timedelta(seconds=int(elapsed))
        return Text("elapsed: " + str(elapsed_delta), style=light_blue_light)


class SpeedColumn(TextColumn):
    _renderable_cache = {}
    _table_column = None

    def __init__(self, *args):
        pass

    def render(self, task):
        if task.speed is None:
            return " "
        else:
            return f"{task.speed:.1f} steps/s"


COLUMNS = [
    BarColumn(bar_width=None),
    "Completed: ",
    TextColumn("[bold magenta]Completed {task.completed}/{task.total}"),
    "[progress.percentage]{task.percentage:>3.0f}%",
    "•",
    "Speed: ",
    SpeedColumn(),
    "•",
    "Remaining: ",
    TimeRemainingColumn(),
    "Elpsed: ",
    TimeElapsedColumn(),
]

progress = Progress(*COLUMNS, transient=False,)

progess_with_description = Progress(
    *([TextColumn("[bold salmon]Task: {task.description}")] + COLUMNS),
    transient=False,
)


def track(iterable, total=None, description="Working...", transient=False):
    """
        Spawns a progress bar to monitor the progress of a for loop over
        an iterable sequence with detailed information.

        Arguments:
            iterable: list or other iterable object
            total: int. Total length of iterable
            description: str. Text to preprend to the progress bar.

        Returs:
            elements of iterable
    """
    description = f"[{orange}]" + description
    columns = [description] + COLUMNS

    if total is None:
        try:
            total = len(iterable)
        except Exception:
            raise ValueError(
                "Could not get total from iterable, pass a total value."
            )

    track_progress = Progress(*columns, transient=transient)

    with track_progress:
        yield from track_progress.track(
            iterable, total=total, description=description,
        )


def create_bash_files(NAME, video_files,bash_folder,winstor_folder,above_ceph_out,BASH_TEMPLATE,BASH_INSERT,make_test_video):

    logger.info(
        f"Found {len(video_files)} videos left to track. Generating bash files."
    )
    for index,video in enumerate(track(video_files)):
        
        # set video path
        winstor_video_path = winstor_folder / video

        # define path for final data to be save out 
        winstor_save_path = '/ceph/sjones/' + '/'.join(above_ceph_out[index].parts[1::]) + '/video/tracking/'
        # local non ceph version for creating data file struct 
        winstor_save_path_local = r'\\'.join(above_ceph_out[index].parts) + r'\video\tracking\\'

        if str(video_files[index]).split('_')[1] == 'task':
            winstor_save_path = winstor_save_path + '/2_task'
            winstor_save_path_local = winstor_save_path_local + '2_task\\'
        elif str(video_files[index]).split('_')[1] == 'presleep':
            winstor_save_path = winstor_save_path + '/1_presleep'
            winstor_save_path_local = winstor_save_path_local + '1_presleep\\'
        elif str(video_files[index]).split('_')[1] == 'postleep':
            winstor_save_path = winstor_save_path + '/3_postsleep'
            winstor_save_path_local = winstor_save_path_local + '3_postsleep\\'
            
        if not os.path.exists(winstor_save_path_local):
            print('creating out path: ' + winstor_save_path_local)
            os.makedirs(winstor_save_path_local)
        
        # create bash file
        bash_content = BASH_INSERT.replace(
            "VIDEO", str(winstor_video_path)
        ).replace("DEST", str(winstor_save_path)).replace("NUMBER",str(index+1))

        BASH_TEMPLATE = BASH_TEMPLATE + bash_content 
        
        if make_test_video:
            BASH_TEMPLATE = BASH_TEMPLATE + 'make test video'

        # write bash file
        bash_path = bash_folder / (NAME + ".sh")
        with open(bash_path, "w") as fout:
            fout.write(BASH_TEMPLATE)

        # replacement strings
        WINDOWS_LINE_ENDING = b'\r\n'
        UNIX_LINE_ENDING = b'\n'
        # relative or absolute file path, e.g.:
        file_path = bash_path
        with open(file_path, 'rb') as open_file:
            content = open_file.read()

        content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)
        with open(file_path, 'wb') as open_file:
            open_file.write(content)
            
        

# set paths

In [20]:
videos_path = r"Z:\projects\sequence_squad\revision_data\emmett_revisions\DLC_video_dump\\"
base_out_path = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"
 

back_video_files = []
back_ceph_out = []  
for avi_file in os.listdir(os.path.join(videos_path,'BACK')):
    if '.avi' in avi_file:
        back_video_files.append(Path(avi_file))
        
        mouse_name = avi_file.split('_')[2]
        if mouse_name == 'AP5':
            date = avi_file.split('_')[5]
            if avi_file.split('_')[4] == 'LR':
                mouse_name= mouse_name.lower() + avi_file.split('_')[4].lower()
            else:
                mouse_name = mouse_name.lower() + avi_file.split('_')[4] 
        else:
            date = avi_file.split('_')[3]
            
        print(mouse_name)
        for file in os.listdir(base_out_path):
            if mouse_name in file:
                for recording in os.listdir(os.path.join(base_out_path,file)):
                    if date in recording:
                        back_ceph_out.append(Path(base_out_path) / file / recording)
                        print(avi_file)
                        print(os.path.join(base_out_path , file , recording))
                        break
                    
        
above_video_files = []
above_ceph_out = []  
for avi_file in os.listdir(os.path.join(videos_path,'ABOVE')):
    if '.avi' in avi_file:
        above_video_files.append(Path(avi_file))
        
        mouse_name = avi_file.split('_')[2]
        if mouse_name == 'AP5':
            date = avi_file.split('_')[5]
            if avi_file.split('_')[4] == 'LR':
                mouse_name= mouse_name.lower() + avi_file.split('_')[4].lower()
            else:
                mouse_name = mouse_name.lower() + avi_file.split('_')[4] 
        else:
            date = avi_file.split('_')[3]
            
        print(mouse_name)
        for file in os.listdir(base_out_path):
            if mouse_name in file:
                for recording in os.listdir(os.path.join(base_out_path,file)):
                    if date in recording:
                        above_ceph_out.append(Path(base_out_path) / file / recording)
                        print(avi_file)
                        print(os.path.join(base_out_path , file , recording))
                        break


## split for AP5 and hippocampus dlc rigs (these setups required different dlc networks dur to lighting)

hpc_back_video_files = []
ap5_back_video_files = []
for file in back_video_files:
    if 'AP5' in str(file):
        ap5_back_video_files.append(Path(file))
    else:
        hpc_back_video_files.append(Path(file))

hpc_back_ceph_out = []
ap5_back_ceph_out = []
for file in back_ceph_out:
    if 'ap5' in str(file):
        ap5_back_ceph_out.append(Path(file))
    else:
        hpc_back_ceph_out.append(Path(file))



ap5L
BACK_task_AP5_1_L_18-11-2024_subclip_5239.4425-7509.437250000001.avi
Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5L_implant1\recording1_18-11-2024
ap5L
BACK_task_AP5_1_L_19-11-2024_subclip_6796.52325-10441.451875.avi
Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5L_implant1\recording2_19-11-2024
ap5R
BACK_task_AP5_2_R_16-11-2024_subclip_5263.412625000001-7341.147499999999.avi
Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5R_implant1\recording1_16-11-2024
ap5R
BACK_task_AP5_2_R_18-11-2024_subclip_6418.634625000001-8541.315375.avi
Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5R_implant1\recording2_18-11-2024
ap5R
BACK_task_AP5_2_R_19-11-2024_subclip_5544.913500000001-9905.22.avi
Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5R_implant1\recording3_19-11-2024
ap5R
BACK_task_AP5_2_R_20-11-2024_subclip_6537.045875000001-10352.360375.avi
Z:\projects\sequence_squad\revision_data\org

## create bash files for running DLC on the cluster

In [ ]:
# /nfs/nhome/live/emmettt/.conda/envs/DEEPLABCUT/bin/python
# ^^ this points it to the right verison of python on the hpc, otherwise it used the miniconda version which is not the same as the conda env

make_test_video = True

# prepare bash files
BASH_TEMPLATE = """#! /bin/bash
"""

## ABOVE
BASH_INSERT = """
echo "running ABOVE tracking NUMBER" 
/nfs/nhome/live/emmettt/.conda/envs/DEEPLABCUT/bin/python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/ABOVE_general-emmett-2024-12-04/config.yaml' '/VIDEO' '/DEST'
"""
create_bash_files('Above',above_video_files,Path(r"Z:\projects\sequence_squad\revision_data\emmett_revisions\DLC_video_dump\hpc\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/ABOVE//"),above_ceph_out,BASH_TEMPLATE,BASH_INSERT,make_test_video)
print('done')

## BACK_AP5
BASH_INSERT = """
echo "running BACK AP5 tracking NUMBER" 
/nfs/nhome/live/emmettt/.conda/envs/DEEPLABCUT/bin/python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/BACK_ap5_rig-emmett-2024-12-04/config.yaml' '/VIDEO' '/DEST'
"""
create_bash_files('Back',ap5_back_video_files,Path(r"Z:\projects\sequence_squad\revision_data\emmett_revisions\DLC_video_dump\hpc\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/BACK//"),ap5_back_ceph_out,BASH_TEMPLATE,BASH_INSERT,make_test_video)
print('done')

## BACK_Hoppocampus
BASH_INSERT = """
echo "running BACK hippocampus tracking NUMBER" 
/nfs/nhome/live/emmettt/.conda/envs/DEEPLABCUT/bin/python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/BACK_hippocampus-emmett-2024-12-10/config.yaml' '/VIDEO' '/DEST'
"""
create_bash_files('Back',hpc_back_video_files,Path(r"Z:\projects\sequence_squad\revision_data\emmett_revisions\DLC_video_dump\hpc\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/BACK//"),hpc_back_ceph_out,BASH_TEMPLATE,BASH_INSERT,make_test_video)
print('done')


## BACK PORTS
BASH_INSERT = """
echo "running BACK PORTS tracking NUMBER" 
/nfs/nhome/live/emmettt/.conda/envs/DEEPLABCUT/bin/python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/revision_data/emmett_revisions/DLC_video_dump/hpc/trained_networks/BACK_PORTS_hpc_rig-emmett-2024-12-04/config.yaml' '/VIDEO' '/DEST'
"""
create_bash_files('Back_Ports',back_video_files,Path(r"Z:\projects\sequence_squad\revision_data\emmett_revisions\DLC_video_dump\hpc\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/revision_data/emmett_revisions/DLC_video_dump/BACK//"),back_ceph_out,BASH_TEMPLATE,BASH_INSERT,make_test_video)
print('done')


2024-11-30 18:13:22.584 | INFO     | __main__:create_bash_files:126 - Found 55 videos left to track. Generating bash files.


Output()

2024-11-30 18:13:32.116 | INFO     | __main__:create_bash_files:126 - Found 21 videos left to track. Generating bash files.


Output()

done


2024-11-30 18:13:36.223 | INFO     | __main__:create_bash_files:126 - Found 21 videos left to track. Generating bash files.


Output()

done


done
